"""

Author: Annam.ai IIT Ropar
Team Name: Ice 'N' Dagger
Team Members: Barun Saha, Bibaswan Das
Leaderboard Rank: 70

"""

# This is the notebook used for making the inferences using the model trained.

In [ ]:
import numpy as np
import pandas as pd
import cv2
import os
import joblib
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.applications import ResNet50, VGG16
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg_preprocess
from tqdm import tqdm

In [ ]:
# Paths
TEST_PATH = '/kaggle/input/soil-classification-part-2/soil_competition-2025/test'
TEST_CSV = '/kaggle/input/soil-classification-part-2/soil_competition-2025/test_ids.csv'
MODEL_PATH = '/kaggle/working/ocsvm_model.pkl'
SCALER_PATH = '/kaggle/working/scaler.pkl'


In [ ]:
# Load test dataframe
test_df = pd.read_csv(TEST_CSV)

In [ ]:
# Load trained One-Class SVM and scaler
ocsvm = joblib.load(MODEL_PATH)
scaler = joblib.load(SCALER_PATH)


In [ ]:
# Load pre-trained feature extractors
resnet_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')
vgg_model = VGG16(weights='imagenet', include_top=False, pooling='avg')

def extract_features_from_image(img_path, resnet_model, vgg_model, img_size=(224, 224)):
    try:
        img = cv2.imread(img_path)
        if img is None:
            return None
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_resized = cv2.resize(img, img_size)

        # Preprocess
        img_resnet = resnet_preprocess(np.expand_dims(img_resized, axis=0))
        img_vgg = vgg_preprocess(np.expand_dims(img_resized, axis=0))

        # Extract
        resnet_features = resnet_model.predict(img_resnet, verbose=0).flatten()
        vgg_features = vgg_model.predict(img_vgg, verbose=0).flatten()

        return np.concatenate([resnet_features, vgg_features])
    except:
        return None

In [ ]:
# Inference
predictions = []
ids = []
for _, row in tqdm(test_df.iterrows(), total=len(test_df)):
    img_id = row['image_id']
    img_path = os.path.join(TEST_PATH, img_id)
    features = extract_features_from_image(img_path, resnet_model, vgg_model)
    if features is not None:
        features_scaled = scaler.transform([features])
        pred = ocsvm.predict(features_scaled)[0]
        label = 1 if pred == 1 else 0
    else:
        label = 0  # default to not soil
    predictions.append(label)
    ids.append(img_id)


In [ ]:
# Submission
submission_df = pd.DataFrame({
    'image_id': ids,
    'label': predictions
})

submission_path = '/kaggle/working/submission.csv'
submission_df.to_csv(submission_path, index=False)

print(f"Submission file saved to: {submission_path}")
